Harika, o zaman **yaptığın tüm süreci ve son eklemeyi tek bir akış hâlinde** özetleyelim. Bu, hem ne yaptığını hem de bundan sonra nasıl kullanacağını net gösterir.

---

# 1️⃣ Ortamı hazırladın (M2 Mac + Python venv)

**Ne yaptın:**

1. Mevcut environment’ı kapattın:

```bash
deactivate
```

2. Temiz bir virtual environment oluşturdun:

```bash
python3 -m venv my_steps_env
source my_steps_env/bin/activate
```

3. Paketleri yükledin ve uyumlu hale getirdin:

```bash
pip install --upgrade pip setuptools wheel
pip install torch torchvision torchaudio      # PyTorch ≥2.6, M2 Metal uyumlu
pip install "numpy<2"                         # NumPy uyumlu
pip install sentence-transformers transformers python-dotenv pinecone grpcio protobuf googleapis-common-protos
pip install openai                             # opsiyonel, artık lokal LLM ile de çalışıyor
```

**Neden:**

* PyTorch ≥2.6 → M2 Mac Metal GPU uyumlu
* NumPy <2 → sentence-transformers uyumu
* Pinecone v3 + gRPC çalışması için protobuf ve grpcio gerekli

---

# 2️⃣ .env dosyası ile Pinecone API key ekledin

```
PINECONE_API_KEY=senin_api_key
```

* Pinecone’a bağlanabilmek için API key şart
* `load_dotenv()` ile script’ine çekiyorsun

---

# 3️⃣ Pinecone index oluşturup kontrol ettin

```python
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

if "rag-demo" not in pc.list_indexes().names():
    pc.create_index(
        name="rag-demo",
        dimension=384,         # BGE-small embedding boyutu
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
```

* Index = veri tabanı, embedding’leri burada saklıyorsun
* Boyut (dimension) embedding modeli ile aynı olmalı

---

# 4️⃣ Embedding modelini yükledin

```python
from sentence_transformers import SentenceTransformer
import torch

device = "mps" if torch.backends.mps.is_available() else "cpu"
model = SentenceTransformer("BAAI/bge-small-en", device=device)
```

* BGE-small → metinleri vektöre (embedding) çeviriyor
* M2 Metal GPU varsa kullanıyor, yoksa CPU fallback

---

# 5️⃣ Dokümanları embed edip Pinecone’a upsert ettin

```python
docs = [("doc1", "FastAPI is a Python web framework."),
        ("doc2", "Docker containers are used for packaging apps.")]

vectors = [(doc_id, model.encode(text).tolist(), {"text": text}) for doc_id, text in docs]
pc.upsert(index_name="rag-demo", vectors=vectors)
```

* Metadata ile metin içeriğini sakladın
* Artık query ile eşleşme bulabilir hale geldi

---

# 6️⃣ Query yaptın ve benzer dokümanları aldın

```python
query = "What is FastAPI?"
q_vec = model.encode(query).tolist()
res = pc.query(index_name="rag-demo", vector=q_vec, top_k=2, include_metadata=True)
print(res)
```

* `score` → dokümanın query ile benzerliği
* `metadata` → orijinal metin

---

# 7️⃣ RAG pipeline: Query → Context → Prompt → LLM response (yeni ekleme)

Sen OpenAI kullanmak yerine **Flan-T5 (seq2seq)** kullandın:

```python
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
llm_pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)

# Context oluştur
context = "\n".join([match['metadata']['text'] for match in res['matches']])
prompt = "Answer based on context:\n" + context + "\nQuestion: " + query

# LLM response
answer = llm_pipe(prompt)[0]['generated_text']

print("\n--- Context from Pinecone ---\n", context)
print("\n--- LLM Answer ---\n", answer)
```

**Ne oldu:**

* Pinecone’dan top_k dokümanları aldın
* Context oluşturup prompt hazırlandı
* Flan-T5 modelinden yanıt üretildi
* Artık **tamamen ücretsiz ve local LLM** ile RAG pipeline çalışıyor

---

# 8️⃣ Bundan sonra nasıl kullanacaksın

1. **FastAPI ile API oluştur:**

   * Kullanıcı query gönderir → embedding → Pinecone → top_k → context → Flan-T5 → yanıt döner

2. **Dockerize et:**

   * Tek script’i container içine koy → RAG service olarak deploy

3. **Dokümanları artır ve test et:**

   * Farklı namespace’ler ile Pinecone limitlerini aşmadan büyüt

---

# 🔹 Özet

* Ortam → hazır ve uyumlu
* Pinecone → vektör depolama ve retrieval
* Embedding → BGE-small
* LLM → Flan-T5 (local, ücretsiz)
* Pipeline → query → embedding → Pinecone → context → prompt → LLM response

Bu noktada pipeline **çalışıyor ve test edilebilir** durumda.
